In [1]:
using Laplacians

objc[16946]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[16946]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[16946]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[16946]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


In [2]:
include("treeSolver.jl")
include("IO.jl")
include("generators.jl")
include("toHighPrecision.jl")
include("verbosePCG.jl")

pcgWorker (generic function with 1 method)

In [3]:
pwd()

"/Users/serbanstan/git/TreePCG/julia"

In [4]:
graphp = "../graphs/xu_2/"

"../graphs/xu_2/"

In [5]:
a = readFromFile(graphp * "graph.mtx");

In [6]:
tree = readFromFile(graphp * "tree.mtx");

In [7]:
x = readFromFile(graphp * "x.vec");

In [8]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);
a.n

10000

In [9]:
@time F = treeSolver(tree)
@time F(b);

  0.300637 seconds (281.74 k allocations: 14.587 MB, 1.72% gc time)
  0.165747 seconds (275.68 k allocations: 8.072 MB, 3.51% gc time)


In [10]:
# a run on standard Float64 data types
numIts = 3000;
@time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=true);
norm(lap(a) * myx64 - b) / norm(b)
insert!(dbg, 1, "$(numIts) iterations")
insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
writeToFile(graphp * "_log_julia_default.txt", dbg)

Working on iteration 10 with a-norm error 274.5391450332328
Working on iteration 20 with a-norm error 257.61555613806087
Working on iteration 30 with a-norm error 242.09503136762052
Working on iteration 40 with a-norm error 224.1936044697305
Working on iteration 50 with a-norm error 202.36958623295897
Working on iteration 60 with a-norm error 182.80137596128887
Working on iteration 70 with a-norm error 170.51423983220502
Working on iteration 80 with a-norm error 154.20697996887696
Working on iteration 90 with a-norm error 134.37735374225082
Working on iteration 100 with a-norm error 122.41199710041806
Working on iteration 110 with a-norm error 109.44195813782292
Working on iteration 120 with a-norm error 97.33072801088048
Working on iteration 130 with a-norm error 88.34379319687596
Working on iteration 140 with a-norm error 80.18333529191834
Working on iteration 150 with a-norm error 71.24441495822695
Working on iteration 160 with a-norm error 63.76420875139084
Working on iteration 170

In [11]:
numIts = 3000
for hp in [64,128,1024]
    set_bigfloat_precision(hp)
    laHP = toHighPrecision(la; precision=hp)
    treeHP = toHighPrecision(tree; precision=hp)
    xHP = toHighPrecision(x; precision=hp);
    bHP = toHighPrecision(b; precision=hp);
    FHP = treeSolver(treeHP)
    
    @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_$(hp).txt", dbg)
end

Working on iteration 10 with a-norm error 273.83432391047256
Working on iteration 20 with a-norm error 257.5598355919738
Working on iteration 30 with a-norm error 240.43788041171706
Working on iteration 40 with a-norm error 222.9682793918019
Working on iteration 50 with a-norm error 200.83033174174665
Working on iteration 60 with a-norm error 181.35016515674445
Working on iteration 70 with a-norm error 162.63090508410124
Working on iteration 80 with a-norm error 145.90057712769155
Working on iteration 90 with a-norm error 131.51993793380643
Working on iteration 100 with a-norm error 114.9011055956396
Working on iteration 110 with a-norm error 102.80993586372539
Working on iteration 120 with a-norm error 93.40045005088396
Working on iteration 130 with a-norm error 81.55177961062468
Working on iteration 140 with a-norm error 73.2677852051251
Working on iteration 150 with a-norm error 64.5710861739224
Working on iteration 160 with a-norm error 57.482162626438
Working on iteration 170 with